# MTGBM

In [2]:
import os
import json
import torch
import random
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from hyperopt import fmin, tpe, hp, Trials
from scipy.stats import pearsonr
import numpy as np
from copy import deepcopy
import lightgbmmt as lgb
import matplotlib.pyplot as plt
import math
current_dir = os.getcwd()


def return_mtgbm_x_y(df_data, tasks):
    y = np.array(df_data[tasks].values)

    auxiliary_data = []
    if use_t_ph_embedding:
        ph = df_data['ph'].values.reshape(-1, 1)
        t = df_data['t'].values.reshape(-1, 1)
        auxiliary_data.append(ph)
        auxiliary_data.append(t)

    if use_mw_logp:
        mw = df_data['mw'].values.reshape(-1, 1)
        logp = df_data['logp'].values.reshape(-1, 1)
        auxiliary_data.append(mw)
        auxiliary_data.append(logp)

    protein_data = np.array(df_data[protein_column].tolist())
    substrate_data = np.array(df_data[substrate_column].tolist())

    x = np.hstack([protein_data, substrate_data] + auxiliary_data)
    return x, y

def return_scores(y_true, y_pred):
    mask = y_true != fill_nan_value
    y_true = y_true[mask]
    y_pred = y_pred[mask]

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    pcc = pearsonr(y_true, y_pred)[0]

    return rmse, mae, r2, pcc

def print_scores(task_scores_dict):
    for task_name in task_names:
        print(f"{task_name}\t RMSE\t MAE\t R2\t PCC\t")

        task_val_scores = task_scores_dict[task_name]['val']
        task_test_scores = task_scores_dict[task_name]['test']

        val_metrics = [f"{np.mean(task_val_scores[metric_name]):.4f}\t" for metric_name in
                       score_names]
        print("Val  " + " ".join(val_metrics))

        test_metrics = [f"{np.mean(task_test_scores[metric_name]):.4f}\t" for metric_name in
                        score_names]
        print("Test " + " ".join(test_metrics))
        print()


def search_best_param(max_evals, params_json_path):
    search_space = {
        "learning_rate": hp.uniform("learning_rate", 0.01, 0.2),
        "max_depth": hp.randint("max_depth", 8, 13),
        "num_leaves": hp.randint("num_leaves", 1000, 4000),
        "min_data_in_leaf": hp.randint("min_data_in_leaf", 10, 40),
        "lambda_l1": hp.uniform("lambda_l1", 0, 1),
        "lambda_l2": hp.uniform("lambda_l2", 0, 1),
        "feature_fraction": hp.uniform("feature_fraction", 0.6, 1),
        "bagging_fraction": hp.uniform("bagging_fraction", 0.6, 1),
        "num_iterations": hp.randint("num_iterations", 1000, 4000),
        'max_bin': hp.randint("max_bin", 50, 300),
    }

    trials = Trials()
    print('start searching params')
    best_params = fmin(fn=search_mtgbm, space=search_space, algo=tpe.suggest, max_evals=max_evals, trials=trials)

    best_params['max_depth'] = int(best_params['max_depth'])
    best_params['num_leaves'] = int(best_params['num_leaves'])
    best_params['num_iterations'] = int(best_params['num_iterations'])
    best_params['min_data_in_leaf'] = int(best_params['min_data_in_leaf'])
    best_params['max_bin'] = int(best_params['max_bin'])

    # 将字典保存为 JSON 文件
    with open(params_json_path, 'w') as json_file:
        json.dump(best_params, json_file)

    return best_params


def search_mtgbm(params):
    print(f'params: {params}')
    temp_params = deepcopy(params)
    temp_params.update({"verbosity": -1, "objective": "custom", "num_labels": num_tasks, "tree_learner": 'serial2', "num_threads": num_threads})
    search_iterations = temp_params.pop("num_iterations")

    kcatkm_rmse_list = []
    for fold_idx, (train_index, val_index) in enumerate(kf.split(train_val_x), start=1):
        print(f'Fold {fold_idx}')
        train_x, val_x = train_val_x[train_index], train_val_x[val_index]
        train_y, val_y = train_val_y[train_index], train_val_y[val_index]

        train_data = lgb.Dataset(train_x, label=train_y)
        val_data = lgb.Dataset(val_x, label=val_y)

        evals_result_mt = {}
        lgb.train(temp_params, train_data, search_iterations, valid_sets=[val_data],
                  fobj=cal_grad, feval=self_kcatkm_rmse, verbose_eval=2000, evals_result=evals_result_mt,
                  callbacks=[lgb.early_stopping(stopping_rounds=500)])

        min_rmse = np.min(np.array(evals_result_mt['valid_0']['rmse_kcatkm']))
        print(f"valid_records min_rmse: {min_rmse:.4f}")
        kcatkm_rmse_list.append(min_rmse)

    valid_rmse_mean = np.mean(kcatkm_rmse_list)
    print(f"[valid rmse mean]: {valid_rmse_mean:.4f}\n")
    return valid_rmse_mean

def self_kcatkm_rmse(preds, train_data):
    labels = torch.tensor(train_data.get_label(), device=device)
    preds = torch.tensor(preds, device=device)

    # extract kcatkm values
    labels = labels.view(num_tasks, -1).T[:, num_tasks - 1]
    preds = preds.view(num_tasks, -1).T[:, num_tasks - 1]

    # mask
    valid_mask = labels != fill_nan_value
    valid_labels = labels[valid_mask]
    valid_preds = preds[valid_mask]

    kcatkm_rmse = torch.sqrt(torch.mean((valid_labels - valid_preds) ** 2))

    return 'rmse_kcatkm', kcatkm_rmse.item(), False


def cal_grad(preds, train_data, ep=0):
    labels = torch.tensor(train_data.get_label(), device=device)
    preds = torch.tensor(preds, device=device)
    labels = labels.view(num_tasks, -1).T
    preds = preds.view(num_tasks, -1).T

    # mask
    valid_mask = labels != fill_nan_value
    grad = torch.zeros_like(preds)
    grad[valid_mask] = preds[valid_mask] - labels[valid_mask]

    # sum
    grad_final = grad.mean(dim=1)

    # Hessian
    grad_flattened = grad.T.flatten()
    hess = torch.ones_like(grad_final)
    hess2 = torch.ones_like(grad_flattened)

    return grad_final.cpu().numpy(), hess.cpu().numpy(), grad_flattened.cpu().numpy(), hess2.cpu().numpy()


# TODO Train model
def train_mtgbm(params):
    temp_params = deepcopy(params)
    temp_params.update({"verbosity": -1, "objective": "custom", "num_labels": num_tasks, "tree_learner": 'serial2', "num_threads": num_threads})
    num_iterations = temp_params.pop("num_iterations")

    task_scores_dict = {task_name: {'val': {name: [] for name in score_names}, 'test': {name: [] for name in score_names}} for task_name in task_names}

    for fold_idx, (train_index, val_index) in enumerate(kf.split(train_val_x), start=1):
        print(f"Fold {fold_idx}")
        # split dataset
        train_x, val_x = train_val_x[train_index], train_val_x[val_index]
        train_y, val_y = train_val_y[train_index], train_val_y[val_index]

        train_data = lgb.Dataset(train_x, label=train_y)

        val_data = lgb.Dataset(val_x, label=val_y)

        # get the best epoch number
        evals_result_mt = {}
        lgb.train(temp_params, train_data, num_iterations, valid_sets=[val_data],
                  fobj=cal_grad, feval=self_kcatkm_rmse, verbose_eval=1000, evals_result=evals_result_mt,
                  callbacks=[lgb.early_stopping(stopping_rounds=500)])
        valid_records = evals_result_mt['valid_0']['rmse_kcatkm']
        min_index = np.argmin(np.array(valid_records))
        print(f"valid_records min_index {min_index}")

        # train model for all scores of validation and test
        train_data = lgb.Dataset(train_x, label=train_y)
        val_data = lgb.Dataset(val_x, label=val_y)
        evals_result_mt = {}
        model = lgb.train(temp_params, train_data, min_index + 1, valid_sets=[val_data],
                          fobj=cal_grad, feval=self_kcatkm_rmse, verbose_eval=1000,
                          evals_result=evals_result_mt)
        model.set_num_labels(num_tasks)

        # validation predict
        val_predicted = model.predict(val_x)
        val_scores = {task_name: return_scores(val_y[:, idx], val_predicted[:, idx]) for idx, task_name in
                      enumerate(task_names)}

        # test predict
        test_predicted = model.predict(test_x)
        test_scores = {task_name: return_scores(test_y[:, idx], test_predicted[:, idx]) for idx, task_name in
                       enumerate(task_names)}

        # record
        for task_name in task_names:
            for score_idx, score_name in enumerate(score_names):
                task_scores_dict[task_name]['val'][score_name].append(val_scores[task_name][score_idx])
                task_scores_dict[task_name]['test'][score_name].append(test_scores[task_name][score_idx])
        print(f"Val  {val_scores} \n Test {test_scores}\n")

    print_scores(task_scores_dict)


# init seed
random_state = 66
random.seed(random_state)
np.random.seed(random_state)
torch.manual_seed(random_state)
torch.cuda.manual_seed(random_state)
torch.cuda.manual_seed_all(random_state)

# config
protein_column,  substrate_column = 'prott5', 'molebert'
input_model = 'mtgbm_km_kcat_kcatkm'
dataset_path = f'{current_dir}/../data_process/dataset/df_all_log_transformed.pkl'
params_json_path = f'{current_dir}/model_dict/{input_model}_params.json'
use_t_ph_embedding = True
use_mw_logp = True
num_threads = 32
search_max_evals = 60
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"Current device is {device}")

# input
score_names = ['rmse', 'mae', 'r2', 'pcc']
task_names = ['logkm', 'logkcat', 'logkcatkm']
num_tasks = len(task_names)
df_input = pd.read_pickle(dataset_path)
fill_nan_value = -100
df_input = df_input.fillna(fill_nan_value)

# split dataset
df_train_val, df_test = train_test_split(df_input, test_size=0.2, random_state=random_state)
kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
train_val_x, train_val_y = return_mtgbm_x_y(df_train_val, task_names)
test_x, test_y = return_mtgbm_x_y(df_test, task_names)

if os.path.exists(params_json_path):
    with open(params_json_path, 'r') as json_file:
        best_params = json.load(json_file)

else:
    best_params = search_best_param(search_max_evals, params_json_path)

print('best_params:', best_params)
print('using km kcat kcatkm resample')
train_mtgbm(best_params)

Current device is cuda:0
start searching params
params: {'bagging_fraction': 0.7186703600474142, 'feature_fraction': 0.6900628341528715, 'lambda_l1': 0.334170267287772, 'lambda_l2': 0.26168029620649047, 'learning_rate': 0.09491687339673566, 'max_bin': 245, 'max_depth': 8, 'min_data_in_leaf': 35, 'num_iterations': 1078, 'num_leaves': 1499}
Fold 1
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[1077]	valid_0's rmse_kcatkm: 2.83449
valid_records min_rmse: 2.8345
Fold 2
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[1078]	valid_0's rmse_kcatkm: 2.74793
valid_records min_rmse: 2.7479
Fold 3
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Best iteration is:
[1078]	valid_0's rmse_kcatkm: 2.72136
valid_records min_rmse: 2.7214
Fold 4
Training until validation scores don't improve for 500 rounds
Did not meet early stopping. Bes